<a href="https://colab.research.google.com/github/PRASANNA-416/EC386-Image-processing-Project/blob/main/EC386_with_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline



from sklearn.model_selection import train_test_split

In [ ]:

from tensorflow import keras
from keras.layers import Conv2D, Input, MaxPool2D,Flatten, Dense, Permute, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.optimizers import Adam
import keras
import dlib
import os.path
from keras.models import Sequential
# from keras.applications.resnet50.ResNet50 import ResNet50
# from keras.applications.resnet50 import Dense
import cv2
import os
from keras.layers import Dropout

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Using Transfer Learning Approach

In [ ]:
from keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [ ]:
# train pyolorus  = /content/gdrive/MyDrive/Mini_Project/Pyrolous/Train
# test pyolorus = /content/gdrive/MyDrive/Mini_Project/Pyrolous/Test
# train Z line = /content/gdrive/MyDrive/Mini_Project/Z-line/Train
# test Z line = /content/gdrive/MyDrive/Mini_Project/Z-line/Test
# train cecum = /content/gdrive/MyDrive/Mini_Project/cecum/Train
#test cecum = /content/gdrive/MyDrive/Mini_Project/cecum/Test

In [ ]:
from tensorflow import keras
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# DO NOT CHANGE THIS CELL
base_model.trainable = False

In [ ]:
# inputs = keras.Input(shape=(224, 224, 3))
# # Separately from setting trainable on the model, we set training to False 
# x = base_model(inputs, training=False)
# x = keras.layers.GlobalAveragePooling2D()(x)

# x = keras.layers.Flatten(name="flatten")(x)
# x = keras.layers.Dense(128,activation = 'relu')(x)
# x = keras.layers.Dense(64,activation = 'relu')(x)
# x = keras.layers.Dense(32,activation = 'relu')(x)
# x = keras.layers.Dropout(0.2)(x)
# x = keras.layers.Dense(64,activation = 'relu')(x)
# x = keras.layers.Dense(128,activation = 'relu')(x)
# x = keras.layers.Dense(32,activation = 'relu')(x)
# x = keras.layers.Dense(64,activation = 'relu')(x)
# x = keras.layers.Dropout(0.1)(x)
# outputs = keras.layers.Dense(3)(x)
# model = keras.Model(inputs, outputs)


def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(3, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes = 3

n_steps = 10 
n_val_steps = 5
n_epochs = 30

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, 3, optim_1, fine_tune=0)


In [ ]:
vgg_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)

In [ ]:
train_it = datagen.flow_from_directory('/content/gdrive/MyDrive/Mini_Project/Train', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=5)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('/content/gdrive/MyDrive/Mini_Project/Test', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='categorical', 
                                      batch_size=5)

Found 132 images belonging to 3 classes.
Found 31 images belonging to 3 classes.


In [ ]:
%%time 

vgg_history = vgg_model.fit(train_it,
                            batch_size= 5 ,
                            epochs= 20,
                            validation_data= valid_it,
                            steps_per_epoch= 10,
                            validation_steps= 3,
                            verbose=1)

Epoch 1/20
10/10 [==============================] - 43s 4s/step - loss: 156.6256 - accuracy: 0.4894 - val_loss: 142.0974 - val_accuracy: 0.4000
Epoch 2/20
10/10 [==============================] - 37s 4s/step - loss: 77.6917 - accuracy: 0.4894 - val_loss: 117.2820 - val_accuracy: 0.4667
Epoch 3/20
10/10 [==============================] - 38s 4s/step - loss: 79.0232 - accuracy: 0.6000 - val_loss: 75.2502 - val_accuracy: 0.6000
Epoch 4/20
10/10 [==============================] - 38s 4s/step - loss: 30.5936 - accuracy: 0.6600 - val_loss: 4.9688 - val_accuracy: 0.8000
Epoch 5/20
10/10 [==============================] - 36s 4s/step - loss: 8.1394 - accuracy: 0.6383 - val_loss: 5.2476 - val_accuracy: 0.6667
Epoch 6/20
10/10 [==============================] - 37s 4s/step - loss: 4.9081 - accuracy: 0.8200 - val_loss: 1.5276 - val_accuracy: 0.8667
Epoch 7/20
10/10 [==============================] - 35s 4s/step - loss: 8.3445 - accuracy: 0.8298 - val_loss: 3.0558 - val_accuracy: 0.9333
Epoch 8/20